In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np
interest_rate=pd.read_csv('/content/gdrive/MyDrive/JPM/Interest Rate/Interest_30_UK.csv',index_col=[0])

In [3]:
import os
path = "/content/gdrive/MyDrive/JPM/UK_data/UK_Speech/"
os.chdir(path)
os.listdir(path)
file_list=[i for i in os.listdir(path) if 'txt' in i]

In [4]:
speech_text=dict()
for i in range(len(file_list)):
  f=open(path+file_list[i],'r',encoding='latin1')
  f1=f.readlines()
  contents=[]
  for line in f1:
    if ('[SECTION]' not in line):
      if len(line.strip().split(' '))>=5:
        if ('Voting for the FOMC monetary policy action were' in line):
          break
        contents.append(line.strip())
  text = ' '.join(contents)
  text=text.replace('â','')
  text=text.replace('ï','')
  text=text.replace('All speeches are available online at www.bankofengland.co.uk/publications/Pages/speeches/default.aspx','')
  text=text.replace('All speeches are available online at www.bankofengland.co.uk/publications/speeches','')
  if len(text.split(' '))>100:
    speech_text[file_list[i][:8]]='speech '+text
  else:
    print(i,len(text.split(' ')),text)
  #print(text)

36 84 Page 1 Inclusive Capitalism Conference: In Conversation with Governor Mark Carney Given at Mansion House, LondonPage 2 Published on 26 June 2015 Governor Mark Carney participates in an on-stage conversation with The Economists Editor-in- Chief Zanny Minton-Beddoes at the Conference on Inclusive Capitalism: The Pathway to Action. The Governor discusses how we can create core financial markets that work fairly and effectively to deliver strong, sustainable and balanced growth. Mark Carney Former Governor, Bank of England from 2013 to 2020 Â©2022 Bank of England
308 32 Page 1 Consumption and Interest Rates - speech by Tim Besley Given at Centre for Economic Policy ResearchPage 2 Published on 19 July 2007 Consumption and Interest Rates Â©2022 Bank of England
455 29 Page 1 Interpreting Monetary Policy - speech by David Miles Speech given at Imperial College, LondonPage 2 Published on 25 February 2010 Interpreting Monetary Policy Â©2022 Bank of England
633 53 Page 1 Momentum in the ho

In [5]:
import os
path = "/content/gdrive/MyDrive/JPM/UK_data/UK_minutes/"
os.chdir(path)
os.listdir(path)
file_list=[i for i in os.listdir(path) if 'txt' in i]

In [6]:
minutes_text=dict()
for i in range(len(file_list)):
  f=open(path+file_list[i],'r',encoding='latin1')
  f1=f.readlines()
  contents=[]
  for line in f1:
    if ('[SECTION]' not in line):
      if len(line.strip().split(' '))>=5:
        if ('Voting for the FOMC monetary policy action were' in line):
          break
        contents.append(line.strip())
  text = ' '.join(contents)
  text=text.replace('â','')
  text=text.replace('ï','')
  text=text.replace('All speeches are available online at www.bankofengland.co.uk/publications/Pages/speeches/default.aspx','')
  text=text.replace('All speeches are available online at www.bankofengland.co.uk/publications/speeches','')
  if len(text.split(' '))>100:
    minutes_text[file_list[i][:8]]='minutes '+text
  else:
    print(i,len(text.split(' ')),text)

8 10 Monetary policy summary and minutes Monetary Policy Committee voting history
12 1 
50 1 
55 21 The Governor invited the Committee to vote on the propositions that: The Committee voted unanimously in favour of all four propositions.


In [7]:
raw_data_speech=pd.DataFrame({'date':speech_text.keys(),'text':speech_text.values()})
raw_data_speech.sort_values(by='date',inplace=True)
raw_data_speech

,date,text
181,19970225,speech Speech by the Governor at the Queen Eli...
157,19970612,speech Lord Mayors Banquet Speech given by...
371,19970624,speech The Sixteenth Mais Lecture Speech gi...
990,19971029,speech The Inflation Target five years on L...
128,19971118,speech Developments in UK financial markets ...
...,...,...
396,20220223,speech The economy and policy trade-offs Sp...
572,20220301,speech The effectiveness of vaccination again...
26,20220330,speech Bank of England Page 1 Technica...
573,20220331,speech Page 1 The road to enhanced payments - ...


In [8]:
raw_data_statement=pd.DataFrame({'date':minutes_text.keys(),'text':minutes_text.values()})
raw_data_statement.sort_values(by='date',inplace=True)
raw_data_statement

,date,text
62,19970716,minutes I Monetary conditions 2 The 12-mon...
64,19970813,minutes I Summary of the analysis presented to...
63,19971015,minutes 1 The meeting took place against the ...
1,19971112,minutes Publication date: 12 November 1997 8 ...
0,19971210,minutes Publication date: 10 December 1997 5 ...
...,...,...
59,20210923,minutes The Bank of EnglandÂs Monetary Policy...
45,20211104,minutes The Bank of EnglandÂs Monetary Policy...
38,20211216,minutes The Bank of EnglandÂs Monetary Policy...
61,20220203,minutes The Bank of EnglandÂs Monetary Policy...


In [9]:
!pip install gensim
import gensim
from gensim.summarization import summarize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 3.4 MB/s 
     |████████████████████████████████| 596 kB 34.2 MB/s 
     |████████████████████████████████| 6.6 MB 37.9 MB/s 
     |████████████████████████████████| 86 kB 5.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [11]:
from tqdm import tqdm,trange


In [12]:
import numpy as np
raw_data_speech['Textrank']=np.nan
for i in trange(len(raw_data_speech)):
  #print(i)
  try:
    short_summary = summarize(raw_data_speech.iloc[i,1],word_count=512)
    short_summary=short_summary.replace('\n',' ').replace('\\','')
    raw_data_speech.iloc[i,-1]=short_summary
  except:
    continue

100%|██████████| 1055/1055 [09:55<00:00,  1.77it/s]


In [13]:
import numpy as np
raw_data_statement['Textrank']=np.nan
for i in trange(len(raw_data_statement)):
  #print(i)
  short_summary = summarize(raw_data_statement.iloc[i,1],word_count=512)
  short_summary=short_summary.replace('\n',' ').replace('\\','')
  raw_data_statement.iloc[i,-1]=short_summary

100%|██████████| 271/271 [02:14<00:00,  2.02it/s]


In [14]:
from transformers import pipeline
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [ ]:
raw_data_speech['Summarization']=np.nan
for i in trange(len(raw_data_speech)):
  text=raw_data_speech.iloc[i,1]
  new_text=""
  start=0
  try:
    while start<len(text.split( )):
      summarized = summarizer(' '.join(text.split(' ')[start:start+700]), min_length=50, max_length=80)
      new_text+=" "+summarized[0]['summary_text']
      start+=700
    print(len(new_text.split(' ')),file_list[i])
    raw_data_speech.iloc[i,-1]=new_text
  except:
    continue
  if (i%100==0 and i!=0):
    raw_data_speech.to_csv('raw_uk_speech.csv')

  0%|          | 0/1055 [00:00<?, ?it/s]

save  0


  0%|          | 1/1055 [00:50<14:39:51, 50.09s/it]

227 19971210_19971106.txt


  0%|          | 2/1055 [01:31<13:14:16, 45.26s/it]

...

 30%|███       | 320/1055 [7:21:55<18:18:26, 89.67s/it]

save  320


 32%|███▏      | 340/1055 [7:56:11<25:10:17, 126.74s/it]

save  340


 34%|███▎      | 356/1055 [8:29:51<27:03:08, 139.33s/it]